In [2]:
!pip install --upgrade --quiet  langchain langchain-community langchain-groq neo4j fastapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.2 MB/s eta 0:00:00


In [3]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 3.3 MB/s eta 0:00:00


In [4]:
!npm install localtunnel


added 22 packages, and audited 23 packages in 4s

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues, run:
  npm audit fix

Run `npm audit` for details.


In [5]:
!pip install ollama

In [18]:
%%writefile app.py

import streamlit as st
import ollama
from langchain_groq import ChatGroq
from neo4j import GraphDatabase
from langchain_community.graphs import Neo4jGraph
from langchain.schema import HumanMessage, AIMessage
import requests
from fastapi import HTTPException

NEO4J_URI="neo4j+s://efe689a5.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="tScJDhi_NBBry7rmL0rVwvkvxaEDfEL2BUK9PqgDCZY"
groq_api_key="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"
news_api="598f444492314398bf47cdf2cc525827"
weather_api="91fcdb9a0ce7eb272ea802e36dc324ae"

llm = ChatGroq(groq_api_key=groq_api_key, model_name="llama-3.1-70b-versatile")
neo4j_driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)


def fetch_weather_data(city, api_key=weather_api):
    geocode_url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
    response = requests.get(geocode_url)

    if response.status_code != 200:
        raise HTTPException(status_code=404, detail="City not found")

    city_data = response.json()
    main=city_data['weather'][0]['main']
    description=city_data['weather'][0]['description']
    temperature=city_data['main']['temp']
    humidity=city_data['main']['humidity']
    wind_speed=city_data['wind']['speed']
    return f"Here the weather of main {city} is {main} having {description}. Here the temperature is {temperature} with humidity {humidity} and wind speed {wind_speed}"

def fetch_news_data(city, api_key=news_api):
    # NewsAPI endpoint for fetching news based on query (city name)
    base_url = "https://newsapi.org/v2/everything"

    params = {
        'q': city,
        'apiKey': api_key,
        'language': 'en',  # Adjust the language as needed
        'pageSize': 5      # Limit to 5 news articles for the response
    }

    response = requests.get(base_url, params=params)

    if response.status_code != 200:
        raise HTTPException(status_code=500, detail="Failed to fetch news data")

    news_data = response.json()
    articles = news_data.get('articles', [])

    if not articles:
        raise HTTPException(status_code=404, detail="No news articles found for the given city")

    # Extract and concatenate all content fields
    combined_content = " ".join([article.get('content', '') for article in articles if article.get('content')])

    return combined_content

# Store initial trip details
if "trip_initialized" not in st.session_state:
    st.session_state.trip_initialized = False

# Get initial trip details once
if not st.session_state.trip_initialized:
    st.title("Personalized Trip Planner")

    # Initial trip information
    st.session_state.location = st.text_input("Location", "Berlin")
    st.session_state.trip_date = st.date_input("Trip Date")
    st.session_state.food_pref = st.selectbox("Preferred Food", ["Local", "International", "Vegetarian", "Any"])
    st.session_state.activity_pref = st.multiselect("Activities", ["Museums", "Shopping", "Sightseeing","Historical Sites","All famous thing there"])
    st.session_state.budget = st.number_input("Budget in dollars", min_value=0, value=100)
    st.session_state.start_location = st.text_input("Starting Location", "Hotel")

    # Confirm trip details to proceed
    if st.button("Confirm Trip Details"):
        st.session_state.trip_initialized = True
        st.write("Trip details saved. You can now start the conversation.")

# Initialize conversation history
if "conversation_history" not in st.session_state:
    st.session_state.conversation_history = [
        AIMessage(content="Hello! I'm your personalized trip planner assistant. Provide me a time to start your journey?")
    ]

#extract the relevant information from the current neo4j database
def extract_information_from_neo4j(user_query, neo4j_driver, llm=llm):
    # Generate Cypher query to fetch relevant information based on user query and conversation history
    # Retrieve up to the last 5 Cypher queries, depending on availability
    if 'cypher_query_history' in st.session_state and st.session_state.cypher_query_history:
      num_queries = min(5, len(st.session_state.cypher_query_history))
      relevant_queries = st.session_state.cypher_query_history[-num_queries:]
    else:
      relevant_queries = []

    prompt = f"""
    Based on the user's query and relevant past Cypher queries, create a new Cypher query to extract meaningful information
    from the Neo4j database. The Cypher query should follow a generalized format that considers user preferences, visited
    places, recent queries, and weather data.

    - **User Query**: "{user_query}"
    - **Relevant Past Cypher Queries**: "{relevant_queries}"

    The Cypher query should be similar to the example format given below without any other comment and make sure not to consider the example as fully its just for illustration how the query should look like. make use of user iput and relevant past queries to generate:

    MATCH (entity1:EntityLabel1)-[relationship:RELATIONSHIP_TYPE]-(entity2:EntityLabel2)
    WHERE entity1.property = $propertyValue
    RETURN entity1, relationship, entity2
    LIMIT 10

   Provide the Cypher query based on the information from the response without any other comment.
    """

    message = HumanMessage(content=prompt)
    # Call llm with a list containing the HumanMessage object
    cypher_query = llm([message]).content

    try:
        # Execute the generated Cypher query on Neo4j
        with neo4j_driver.session() as session:
            result = session.run(cypher_query)
            extracted_info = [record.data() for record in result]

        # Check if results exist, else return an empty list
        if not extracted_info:
            return []

        return extracted_info

    except Exception as e:
        # Log the error (optional), and return an empty list
        return []

# Function to generate trip response
def generate_trip_response(user_input, conversation_history, neo4j_driver=neo4j_driver, llm=llm):
    """
    Generates a trip response by combining the user query, conversation history,
    one-time collected data from session state, and extracted information from Neo4j.

    Parameters:
        user_input (str): The latest query or message from the user.
        conversation_history (list): List of dictionaries with the conversation history.
        extracted_info (list): Information extracted from Neo4j based on the user's context and query.

    Returns:
        str: The generated response.
    """
    # Step 1: Extract relevant information from Neo4j using the user query and conversation history
    extracted_info = extract_information_from_neo4j(user_input, neo4j_driver, llm)
    weather_data = fetch_weather_data(st.session_state.get('location', 'Not specified'))
    news_data = fetch_news_data(st.session_state.get('location', 'Not specified'))

    # Construct the prompt using session state values for one-time data
    prompt = f"""
    You are a travel assistant helping the user plan their trip. Use the following details to answer the user's query:

    - **Location**: {st.session_state.get('location', 'Not specified')}
    - **Trip Date**: {st.session_state.get('trip_date', 'Not specified')}
    - **Food Preference**: {st.session_state.get('food_pref', 'Not specified')}
    - **Activities Preferred**: {", ".join(st.session_state.get('activity_pref', []))}
    - **Budget**: ${st.session_state.get('budget', 'Not specified')}
    - **Starting Location**: {st.session_state.get('start_location', 'Not specified')}
    - **Weather**: {weather_data}
    - **Latest News in Location**: {news_data}

    Here make sure that the news data is appropriately based on the context of the trip or else don't consider it.
    Additional Information from User's Profile:
    {extracted_info if extracted_info else 'No additional information available.'}

    User's Conversation History:
    {conversation_history}

    Now, based on all this information, respond to the following user query:
    "{user_input}"
    Here provide more weightage to the recent and conversation history and extracted data.
    Here is an example of how the response should be:
    'User: Hi, I'd like to plan a one-day trip in Berlin.
    System: Great! Let’s get started. What day are you planning for, and what time do you want
    to start and end your day?
    User: I'll be visiting on the 15th of June. I want to start at 8 AM and finish by 8 PM.
    System: Noted. Could you tell me your interests? For example, do you like historical sites,
    museums, shopping, or food experiences?
    User: I like history and local food.
    System: Excellent! What's your budget for the day?
    User: I want to keep it aSordable, ideally under 100 Euros.
    System: Perfect! Where would you like to start your day? You can provide your hotel or any
    specific location, or we can start from the first attraction itself.
    User: I'm staying at Hotel Berlin Central.
    System: Got it! Here is an initial itinerary for your day in Berlin, starting from Hotel Berlin
    Central:
    • Start Point: Hotel Berlin Central to Brandenburg Gate (8:00 AM - 9:00 AM)
    o Travel Method: Walk (15 minutes)
    o Activity: Visit the iconic Brandenburg Gate and learn about its historical
    significance.
    • Second Stop: Holocaust Memorial (9:15 AM - 10:00 AM)
    o Activity: Walk through the memorial dedicated to the victims of the
    Holocaust.
    • Third Stop: Reichstag Building (10:15 AM - 11:30 AM)
    o Activity: Visit the Reichstag Building and, if you’d like, head up to the glass
    dome for panoramic views of the city.
    o Note: Pre-booking required for the dome visit (free).
    • Lunch: Curry 36 (12:00 PM - 12:45 PM)
    o Cuisine: Enjoy Berlin's famous Currywurst.
    o Cost: Approx. 7 Euros
    • Fourth Stop: Museum Island (1:15 PM - 3:00 PM)
    o Activity: Visit Pergamon Museum or Neues Museum. Entry fees around 12-15
    Euros per museum.
    • Fifth Stop: Berliner Dom (3:15 PM - 4:00 PM)
    o Activity: Explore the Berlin Cathedral, enjoy the architecture, and climb to
    the top for a beautiful view.
    • CoBee Break: Local Café near Alexanderplatz (4:15 PM - 4:45 PM)
    o Cost: Approx. 5 Euros
    • Sixth Stop: East Side Gallery (5:15 PM - 6:15 PM)
    o Activity: Walk along the East Side Gallery, the longest remaining section of
    the Berlin Wall, covered in murals by artists from around the world.
    • Dinner: Prater Garten (6:45 PM - 7:45 PM)
    o Cuisine: Traditional German beer garden experience.
    o Cost: Approx. 20 Euros
    System: Here is the updated itinerary with costs and optimized travel:
    1. Start Point: Hotel Berlin Central to Brandenburg Gate (8:00 AM - 9:00 AM)
    o Travel Method: Walk (15 minutes)
    2. Second Stop: Holocaust Memorial (9:15 AM - 10:00 AM)
    3. Third Stop: Reichstag Building (10:15 AM - 11:30 AM)
    4. Lunch: Curry 36 (12:00 PM - 12:45 PM)
    5. Fourth Stop: Museum Island (1:15 PM - 3:00 PM)
    6. Fifth Stop: Berliner Dom (3:15 PM - 4:00 PM)
    7. CoBee Break: Local Café near Alexanderplatz (4:15 PM - 4:45 PM)
    8. Sixth Stop: East Side Gallery (5:15 PM - 6:15 PM)
    9. Dinner: Prater Garten (6:45 PM - 7:45 PM)
    Total Estimated Cost: Approx. 59 Euros
    System: The weather forecast for the 15th of June looks sunny with mild temperatures,
    perfect for walking tours. I recommend wearing comfortable shoes and carrying a water
    bottle.'
    Here consider the budget we have in dollars every time and give the budget distribution during trip planning.
    Also for any other user input apart from the trip planner provide them a response appropriately.
    Provide a detailed response considering all preferences, and if possible, suggest personalized itinerary items based on extracted information and past preferences.
    """

    # Prepare the message format for Ollama
    #messages = conversation_history + [{"role": "user", "content": prompt}

    message = AIMessage(content=prompt)
    llm_response = llm([message]).content # Call llm with a list containing the HumanMessage object
    return llm_response


# Function to generate Cypher query using ChatGroq
def generate_cypher_query(llm_response, llm=llm):
    if 'cypher_query_history' in st.session_state and st.session_state.cypher_query_history:
      num_queries = min(5, len(st.session_state.cypher_query_history))
      relevant_queries = st.session_state.cypher_query_history[-num_queries:]
    else:
      relevant_queries = []
    user_id = st.session_state.get('session_id', 'default_user')
    prompt = f"""
    You are a graph database assistant. The user response is as follows: "{llm_response}".
    You also have past query data as follows: "{relevant_queries}".
    Extract meaningful information from this response as Entity-Relationship-Entity triplets in the format:
    (User, RELATIONSHIP, Detail). Then convert this into a Cypher query that updates the Neo4j database.
    You can also make connections with past queries if possible

    Example Triplet: ("User", "PLANS_TO_VISIT", "Berlin")

    Format the Cypher query like:
    MERGE (a:User {{name: '{user_id}'}})
    MERGE (b:Detail {{name: 'Extracted detail'}})
    MERGE (a)-[:RELATIONSHIP]->(b)

    Provide the Cypher query based on the information from the response without any other comment.
    """
    # Create a HumanMessage object with the prompt
    message = HumanMessage(content=prompt)
    cypher_query = llm([message]).content # Call llm with a list containing the HumanMessage object
    return cypher_query

# Store response in Neo4j
def store_memory_in_neo4j(cypher_query, neo4j_driver=neo4j_driver):
    with neo4j_driver.session() as session:
        session.run(cypher_query)

# Display chat history
for message in st.session_state.conversation_history:
    if isinstance(message, AIMessage):
        with st.chat_message("AI"):
            st.markdown(message.content)
    elif isinstance(message, HumanMessage):
        with st.chat_message("Human"):
            st.markdown(message.content)

# User input for new query
user_query = st.chat_input("Type a message...")
if user_query is not None and user_query.strip() != "":
    st.session_state.conversation_history.append(HumanMessage(content=user_query))

    with st.chat_message("Human"):
        st.markdown(user_query)

    with st.chat_message("AI"):
        try:
            response = generate_trip_response(user_query, st.session_state.conversation_history)
            st.markdown(response)
            st.session_state.conversation_history.append(AIMessage(content=response))
            # Generate Cypher query to update Neo4j
            cypher_query = generate_cypher_query(response)
            store_memory_in_neo4j(cypher_query, neo4j_driver)
            # Append the generated Cypher query to cypher_query_history for later retrieval
            if 'cypher_query_history' not in st.session_state:
                st.session_state.cypher_query_history = []
            st.session_state.cypher_query_history.append(cypher_query)

        except Exception as e:
            #st.error(f"Error processing query: {e}")
            pass


Overwriting app.py


In [19]:
!streamlit run app.py &>/content/logs.txt & !npx localtunnel --port 8501 & curl ipv4.icanhazip.com

/bin/bash: line 1: !npx: command not found
34.148.187.75


In [20]:
!npx localtunnel --port 8501

your url is: https://ninety-hands-beg.loca.lt
^C
